# 19. RNN with Hamlet

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

from sklearn.feature_extraction.text import CountVectorizer

import random
import numpy as np

## 19.1 Preparing Data

In [2]:
nltk.download("gutenberg")
nltk.download("stopwords")

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\slcf\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\slcf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
raw = nltk.corpus.gutenberg.raw("shakespeare-hamlet.txt")
print(raw[:500])

[The Tragedie of Hamlet by William Shakespeare 1599]


Actus Primus. Scoena Prima.

Enter Barnardo and Francisco two Centinels.

  Barnardo. Who's there?
  Fran. Nay answer me: Stand & vnfold
your selfe

   Bar. Long liue the King

   Fran. Barnardo?
  Bar. He

   Fran. You come most carefully vpon your houre

   Bar. 'Tis now strook twelue, get thee to bed Francisco

   Fran. For this releefe much thankes: 'Tis bitter cold,
And I am sicke at heart

   Barn. Haue you had quiet Guard?
  Fran. Not


## 19.2 Char to Dic

In [4]:
char2index = {}
index2char = []

In [5]:
for char in raw :
    if char not in char2index.keys() :
        char2index[char] = len(char2index)
        index2char.append(char)

In [6]:
char2index

{'\n': 27,
 ' ': 4,
 '!': 64,
 '&': 43,
 "'": 39,
 '(': 61,
 ')': 62,
 ',': 48,
 '-': 54,
 '.': 32,
 '1': 23,
 '5': 24,
 '9': 25,
 ':': 42,
 ';': 58,
 '?': 40,
 'A': 28,
 'B': 35,
 'C': 38,
 'D': 55,
 'E': 34,
 'F': 36,
 'G': 51,
 'H': 12,
 'I': 49,
 'K': 46,
 'L': 45,
 'M': 52,
 'N': 41,
 'O': 56,
 'P': 31,
 'Q': 59,
 'R': 53,
 'S': 19,
 'T': 1,
 'V': 63,
 'W': 18,
 'Y': 47,
 'Z': 66,
 '[': 0,
 ']': 26,
 'a': 6,
 'b': 16,
 'c': 29,
 'd': 8,
 'e': 3,
 'f': 11,
 'g': 7,
 'h': 2,
 'i': 9,
 'j': 65,
 'k': 20,
 'l': 14,
 'm': 13,
 'n': 33,
 'o': 10,
 'p': 22,
 'q': 50,
 'r': 5,
 's': 21,
 't': 15,
 'u': 30,
 'v': 44,
 'w': 37,
 'x': 57,
 'y': 17,
 'z': 60}

In [7]:
len(char2index)

67

In [8]:
char2vec = {}
eye = np.eye(len(char2index))

In [9]:
for item in char2index.keys() :
    char2vec[item] = eye[char2index[item],:]

In [10]:
char2vec['a']

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
data = np.array([char2vec[char] for char in raw])

## 19.3 Define Model

In [12]:
# Parameters:	
# input_size – The number of expected features in the input x
# hidden_size – The number of features in the hidden state h
# num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two RNNs together to form a stacked RNN, with the second RNN taking in outputs of the first RNN and computing the final results. Default: 1
# nonlinearity – The non-linearity to use. Can be either ‘tanh’ or ‘relu’. Default: ‘tanh’
# bias – If False, then the layer does not use bias weights b_ih and b_hh. Default: True
# batch_first – If True, then the input and output tensors are provided as (batch, seq, feature). Default: False
# dropout – If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
# bidirectional – If True, becomes a bidirectional RNN. Default: False

In [13]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
#         self.rnn = nn.GRU(input_size,hidden_size,num_layers)
        self.rnn = nn.RNN(input_size,hidden_size,num_layers, dropout=0.5)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        out,hidden = self.rnn(input.view(1,1,-1),hidden)
        out = self.fc(out.view(1,-1))
        return out,hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.num_layers, 1, self.hidden_size)).cuda()
        return hidden

In [14]:
model = RNN(len(data[0]), 500, len(data[0]), 1).cuda()

## 19.4 Model Training

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

In [16]:
step = 100
num_epochs = 5

In [17]:
for epoch in range(num_epochs):
    
    sp = list(range(0, len(data) - 2 * step, step))
    sp = np.add(sp, random.randint(0, step))
    random.shuffle(sp)
    
    for i in range(len(sp)) :
    
        hidden = model.init_hidden()

        cost = 0

        for pos in range(sp[i], sp[i] + step):
            X = Variable(torch.from_numpy(data[pos]).type(torch.FloatTensor)).cuda()
            y = torch.from_numpy(data[pos+1]).cuda()
            _, y = y.max(dim=0)

            pre, hidden = model(X,hidden)
            cost += loss(pre,Variable(y).cuda())

        cost.backward()
        
        nn.utils.clip_grad_norm(model.parameters(), 5)
        
        optimizer.step()

        if (i + 1) % 100 == 0 :
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f'
                     %(epoch+1, num_epochs, i + 1, len(sp), cost.data[0]))

Epoch [1/5], Iter [100/1627] Loss: 321.3247
Epoch [1/5], Iter [200/1627] Loss: 289.4613
Epoch [1/5], Iter [300/1627] Loss: 252.1387
Epoch [1/5], Iter [400/1627] Loss: 227.3930
Epoch [1/5], Iter [500/1627] Loss: 241.5024
Epoch [1/5], Iter [600/1627] Loss: 226.0522
Epoch [1/5], Iter [700/1627] Loss: 248.2022
Epoch [1/5], Iter [800/1627] Loss: 245.2936
Epoch [1/5], Iter [900/1627] Loss: 240.0690
Epoch [1/5], Iter [1000/1627] Loss: 236.2115
Epoch [1/5], Iter [1100/1627] Loss: 234.6044
Epoch [1/5], Iter [1200/1627] Loss: 215.2743
Epoch [1/5], Iter [1300/1627] Loss: 200.2270
Epoch [1/5], Iter [1400/1627] Loss: 246.0686
Epoch [1/5], Iter [1500/1627] Loss: 235.5276
Epoch [1/5], Iter [1600/1627] Loss: 205.3211
Epoch [2/5], Iter [100/1627] Loss: 216.8372
Epoch [2/5], Iter [200/1627] Loss: 235.4823
Epoch [2/5], Iter [300/1627] Loss: 223.6393
Epoch [2/5], Iter [400/1627] Loss: 237.7616
Epoch [2/5], Iter [500/1627] Loss: 207.4575
Epoch [2/5], Iter [600/1627] Loss: 256.2497
Epoch [2/5], Iter [700/16

In [22]:
start_num = 1
text = index2char[start_num]

model.eval()
hidden = model.init_hidden()

X_test = Variable(torch.from_numpy(data[start_num]).type(torch.FloatTensor)).cuda()
    
for i in range(500) :

    pre, hidden = model(X_test, hidden)

    temp = pre.cpu().data.numpy()[0]

    best_5 = np.argsort(temp)[::-1][:5]
    
    temp = np.exp(temp[best_5])
    
    temp = temp / temp.sum()
    
    pre = np.random.choice(best_5, 1, p = temp)[0]
    
    curr_char = index2char[pre]
    
    text += curr_char
    
    X_test = Variable(torch.from_numpy(char2vec[curr_char]).type(torch.FloatTensor)).cuda()
    
print("* Generated Text : \n", text)

* Generated Text : 
 To se senessent an yeur Soust to saue to thit?
Y that
In his all and at iull in well as as a welf a kay
    Ham. He laugh with or most as yeur or serus as you mase seet me wist as the Sould,
and to and thee say tray if heat, and, are my Lord

   Kin. Nit as a mailes of whit so me that a digg our ould tit wall time at in he gaue tham se for and as

   Ham. The downe a day

   Kin.
Tre aids is a sighines and seaue me his a selfar a denter and seauen as the wherese of tine,
Tis gron thes an was a mu
